# (d.i)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [3]:
train = pd.read_csv("Q2_Train_Data.csv")
train_emotion = train['emotion']
train_pixels = train['pixels'].str.split(' ', expand = True)
train_pixels = train_pixels.apply(pd.to_numeric)

In [4]:
vali = pd.read_csv("Q2_Validation_Data.csv")
vali_emotion = vali['emotion']
vali_pixels = vali['pixels'].str.split(' ', expand=True)
vali_pixels = vali_pixels.apply(pd.to_numeric)

In [5]:
test = pd.read_csv("Q2_Test_Data.csv")
test_emotion = test['emotion']
test_pixels = test['pixels'].str.split(' ', expand=True)
test_pixels = test_pixels.apply(pd.to_numeric)

In [6]:
# Preprocessing: Normalize the images.
train_pixels = (train_pixels / 255) - 0.5
vali_pixels = (vali_pixels / 255) - 0.5
test_pixels = (test_pixels / 255) - 0.5

In [7]:
train_pixels_cnn = train_pixels.values.reshape((-1,48,48,1))
vali_pixels_cnn = vali_pixels.values.reshape((-1,48,48,1))
test_pixels_cnn = test_pixels.values.reshape((-1,48,48,1))

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval
from keras import backend
from keras.layers import Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import time

Using TensorFlow backend.


In [9]:
def optimize_cnn(hyperparameter):
    backend.clear_session()
    cnn_model = Sequential()

    for layer_list in hyperparameter['conv2d_config']:
        for layer_size in layer_list:
            cnn_model.add(Conv2D(layer_size, kernel_size=3, padding='same', strides=hyperparameter['stride_size'], activation='relu'))
        cnn_model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
        cnn_model.add(Dropout(hyperparameter['dropout_prob']))

    cnn_model.add(Flatten())
    cnn_model.add(Dense(512, activation='relu'))
    cnn_model.add(Dense(7, activation='softmax'))

    cnn_model.compile(optimizer=hyperparameter['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'],)

    start = time.time()
    cnn_history = cnn_model.fit(train_pixels_cnn, to_categorical(train_emotion), epochs=20, batch_size=256, verbose=0)
    end = time.time()

    performance = cnn_model.evaluate(vali_pixels_cnn, to_categorical(vali_emotion), verbose=0)

    print("-----------------------------------------------------------------------------------------")
    print("Hyperparameters: ", (hyperparameter))
    print("Accuracy on training sets: %.4f" % (cnn_history.history['accuracy'][-1]))
    print("Accuracy on validation sets: %.4f" % (performance[1]))
    print("Running time: %.4f s" % (end-start))
    print("Number of parameters: %d" % (cnn_model.count_params()))
    print("-----------------------------------------------------------------------------------------")


    # We want to minimize loss i.e. negative of accuracy
    return({"status": STATUS_OK, "loss": -1*performance[1], "model":cnn_model})

# Define search space for hyper-parameters
space = {
    # The convolution layers and sizes
    'conv2d_config': hp.choice('conv2d_config', [[[64],[128]],[[32,32],[64,64]],[[32],[64,64],[32]]]),
    # The stride_size for convolutions:
    'stride_size': hp.choice('stride_size', [1, 2, 3]),
    # Uniform distribution in finding appropriate dropout values
    'dropout_prob': hp.uniform('dropout_prob', 0.1, 0.35),
    # Choice of optimizer 
    'optimizer': hp.choice('optimizer', ['Adam','sgd']),
}

trials = Trials()

# Find the best hyperparameters
best = fmin(
        optimize_cnn,
        space,
        algo=tpe.suggest,
        trials=trials,
        max_evals=5,
    )


-----------------------------------------------------------------------------------------                              
Hyperparameters:                                                                                                       
{'conv2d_config': ((32,), (64, 64), (32,)), 'dropout_prob': 0.18339892017353193, 'optimizer': 'Adam', 'stride_size': 2}
Accuracy on training sets: 0.5313                                                                                      
Accuracy on validation sets: 0.4943                                                                                    
Running time: 145.1889 s                                                                                               
Number of parameters: 94695                                                                                            
-----------------------------------------------------------------------------------------                              
----------------------------------------

# (d.ii)

In [10]:
best_hyper_cnn = space_eval(space,best)
cnn_tuned = optimize_cnn(best_hyper_cnn)
performance_cnn = cnn_tuned['model'].evaluate(test_pixels_cnn, to_categorical(test_emotion), verbose=0)
print("Accuracy on testing sets: ", performance_cnn[1])

-----------------------------------------------------------------------------------------
Hyperparameters:  {'conv2d_config': ((64,), (128,)), 'dropout_prob': 0.18230524602612286, 'optimizer': 'Adam', 'stride_size': 2}
Accuracy on training sets: 0.6648
Accuracy on validation sets: 0.5511
Running time: 278.7731 s
Number of parameters: 668423
-----------------------------------------------------------------------------------------
Accuracy on testing sets:  0.5533574819564819
